In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from tqdm import tqdm
import os

In [ ]:
!pip install sentence-transformers chromadb pandas openpyxl tqdm

In [ ]:
def load_embedding_model(model_name="BAAI/bge-small-en-v1.5"):
    """
    Load BGE embedding model from HuggingFace.
    
    Available BGE models:
    - BAAI/bge-small-en-v1.5 (fastest, 384 dimensions)
    - BAAI/bge-base-en-v1.5 (balanced, 768 dimensions)
    - BAAI/bge-large-en-v1.5 (best quality, 1024 dimensions)
    
    Args:
        model_name: HuggingFace model identifier
    
    Returns:
        SentenceTransformer model
    """
    print(f"Loading embedding model: {model_name}")
    model = SentenceTransformer(model_name)
    print(f" Model loaded successfully!")
    print(f"  Embedding dimension: {model.get_sentence_embedding_dimension()}")
    return model

In [ ]:
def load_excel_data(excel_file):
    """Load policy data from Excel file."""
    try:
        df = pd.read_excel(excel_file, sheet_name='Policies')
        print(f" Loaded {len(df)} rows from {excel_file}")
        print(f"  Columns: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"Error loading Excel: {e}")
        return None

In [ ]:
def initialize_chromadb(persist_directory="./chroma_db", collection_name="policies"):
    """
    Initialize ChromaDB client and create/get collection.
    
    Args:
        persist_directory: Local directory to persist the database
        collection_name: Name of the collection
    
    Returns:
        ChromaDB collection object
    """
    # Create persist directory if it doesn't exist
    os.makedirs(persist_directory, exist_ok=True)
    
    # Initialize ChromaDB client with persistence
    client = chromadb.PersistentClient(path=persist_directory)
    
    print(f"ChromaDB initialized at: {persist_directory}")
    
    # Get or create collection
    try:
        # Try to get existing collection
        collection = client.get_collection(name=collection_name)
        print(f" Retrieved existing collection: {collection_name}")
        print(f"  Current documents: {collection.count()}")
    except:
        # Create new collection if doesn't exist
        collection = client.create_collection(
            name=collection_name,
            metadata={"description": "UF Policy documents with embeddings"}
        )
        print(f" Created new collection: {collection_name}")
    
    return collection

In [ ]:
def create_embeddings_batch(model, texts, batch_size=32):
    """
    Create embeddings for a list of texts in batches.
    
    Args:
        model: SentenceTransformer model
        texts: List of text strings
        batch_size: Number of texts to process at once
    
    Returns:
        List of embeddings
    """
    embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Creating embeddings"):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.extend(batch_embeddings.tolist())
    
    return embeddings

In [ ]:
def store_in_chromadb(collection, df, embeddings, include_types=None):
    """
    Store documents with embeddings in ChromaDB.
    
    Args:
        collection: ChromaDB collection
        df: Pandas DataFrame with policy data
        embeddings: List of embeddings
        include_types: List of policy types to include (None = all)
    
    Returns:
        Number of documents stored
    """
    # Filter by type if specified
    if include_types:
        df = df[df['type'].isin(include_types)]
        print(f"Filtering by types: {include_types}")
        print(f"Documents after filtering: {len(df)}")
    
    # Prepare data for ChromaDB
    documents = []
    metadatas = []
    ids = []
    filtered_embeddings = []
    
    for idx, row in df.iterrows():
        # Create unique ID
        doc_id = f"{row['title']}_{row['chunk_number']}".replace(" ", "_")
        
        # Document text
        documents.append(row['content'])
        
        # Metadata
        metadatas.append({
            'title': str(row['title']),
            'category': str(row['category']),
            'type': str(row['type']),
            # 'link': str(row['link']),
            # 'chunk_number': int(row['chunk_number']),
            # 'total_chunks': int(row['total_chunks']),
            # 'chunk_size': int(row['chunk_size'])
        })
        
        # IDs
        ids.append(f"{doc_id}_{idx}")
        
        # Corresponding embedding
        filtered_embeddings.append(embeddings[idx])
    
    # Store in ChromaDB in batches
    batch_size = 100
    total_stored = 0
    
    for i in tqdm(range(0, len(documents), batch_size), desc="Storing in ChromaDB"):
        batch_end = min(i + batch_size, len(documents))
        
        collection.add(
            documents=documents[i:batch_end],
            embeddings=filtered_embeddings[i:batch_end],
            metadatas=metadatas[i:batch_end],
            ids=ids[i:batch_end]
        )
        
        total_stored += (batch_end - i)
    
    print(f"Stored {total_stored} documents in ChromaDB")
    return total_stored

In [ ]:
def query_chromadb(collection, model, query_text, n_results=5, filter_by_type=None):
    """
    Query ChromaDB with a text query.
    
    Args:
        collection: ChromaDB collection
        model: SentenceTransformer model for query embedding
        query_text: Query string
        n_results: Number of results to return
        filter_by_type: Filter results by policy type (optional)
    
    Returns:
        Query results
    """
    # Create query embedding
    query_embedding = model.encode([query_text])[0].tolist()
    
    # Prepare filter
    where_filter = None
    if filter_by_type:
        where_filter = {"type": {"$eq": filter_by_type}}
    
    # Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        where=where_filter
    )
    
    return results

In [24]:
def query_chromadb(collection, model, query_text, n_results=5, filter_by_type=None):
    """
    Query ChromaDB with a text query.
    
    Args:
        collection: ChromaDB collection
        model: SentenceTransformer model for query embedding
        query_text: Query string
        n_results: Number of results to return
        filter_by_type: Filter results by policy type (optional)
    
    Returns:
        Query results
    """
    # Create query embedding
    query_embedding = model.encode([query_text])[0].tolist()
    
    # Prepare filter
    where_filter = None
    if filter_by_type:
        where_filter = {"type": {"$eq": filter_by_type}}
    
    # Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        where=where_filter
    )
    
    return results



In [ ]:
def display_results(results):
    """Display query results in a readable format."""
    print("\n" + "="*80)
    print("QUERY RESULTS")
    print("="*80)
    
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ), 1):
        print(f"\n Result {i} (Distance: {distance:.4f})")
        print(f"Title: {metadata['title']}")
        print(f"Category: {metadata['category']} | Type: {metadata['type']}")
        # print(f"Chunk: {metadata['chunk_number']}/{metadata['total_chunks']}")
        # print(f"Link: {metadata['link']}")
        print(f"\nContent Preview:")
        print(f"{doc[:300]}...")
        print("-"*80)



In [ ]:
def main_pipeline(excel_file, 
                  persist_directory="./chroma_db",
                  collection_name="policies",
                  model_name="BAAI/bge-small-en-v1.5",
                  include_types=None,
                  overwrite=False):
    """
    Main pipeline to load data, create embeddings, and store in ChromaDB.
    
    Args:
        excel_file: Path to Excel file
        persist_directory: ChromaDB storage directory
        collection_name: Name of ChromaDB collection
        model_name: BGE model name
        include_types: List of types to include (e.g., ['Policy', 'Regulation'])
        overwrite: If True, delete existing collection and create new one
    
    Returns:
        collection: ChromaDB collection object
        model: Embedding model
    """

    
    # Step 2: Load embedding model
    print(f"\nStep 2: Loading embedding model...")
    model = load_embedding_model(model_name)
    
    # Step 3: Initialize ChromaDB
    print(f"\nStep 3: Initializing ChromaDB...")
    if overwrite:
        client = chromadb.PersistentClient(path=persist_directory)
        try:
            client.delete_collection(name=collection_name)
            print(f" Deleted existing collection: {collection_name}")
        except:
            pass
    
    collection = initialize_chromadb(persist_directory, collection_name)
    

    
    print("\n" + "="*80)
    print(" PIPELINE COMPLETE!")
    print(f" Total documents in collection: {collection.count()}")
    print("="*80)
    
    return collection, model

In [31]:
# 1. Basic usage - store all policies
# 3. Use different BGE model
collection, model = main_pipeline(
    excel_file="policies_data.xlsx",
    model_name="BAAI/bge-base-en-v1.5"  # Better quality
)


Step 2: Loading embedding model...
   🤗 Loading model from HuggingFace: BAAI/bge-base-en-v1.5

Step 3: Initializing ChromaDB...
✓ ChromaDB initialized at: ./chroma_db
✓ Retrieved existing collection: policies
  Current documents: 1887

✓ PIPELINE COMPLETE!
✓ Total documents in collection: 1887


In [32]:
# 5. Query with type filter
results = query_chromadb(
    collection=collection,
    model=model,
    query_text="leave and absence policy",
    n_results=3,
    filter_by_type="Policy"  # Only search Policy type
)

display_results(results)


QUERY RESULTS

📄 Result 1 (Distance: 0.5600)
Title: Working Safely and Maintaining Workplace Health Standards
Category: Human Resources | Type: Policy

Content Preview:
. Employees should not return to work until symptoms have improved. Employees must follow all department reporting procedures for absences, including accurately reporting their work time and absence from the office in myUFL. During the absence, employees may work from home, if approved, and shall us...
--------------------------------------------------------------------------------

📄 Result 2 (Distance: 0.5679)
Title: Furlough Policy (includes UFF Faculty MOU)
Category: Human Resources | Type: Policy

Content Preview:
. Policy Statement A furlough is a mandatory unpaid partial or full leave of absence from work. The University shall consider the availability and feasibility of other cost‐saving measures before implementing furloughs...
--------------------------------------------------------------------------------

📄

In [33]:
results

{'ids': [['Working_Safely_and_Maintaining_Workplace_Health_Standards_3_1520',
   'Furlough_Policy_(includes_UFF_Faculty_MOU)_2_1853',
   'Paid_Family_Leave_7_1595']],
 'embeddings': None,
 'documents': [['. Employees should not return to work until symptoms have improved. Employees must follow all department reporting procedures for absences, including accurately reporting their work time and absence from the office in myUFL. During the absence, employees may work from home, if approved, and shall use accrued sick leave or leave without pay if unable to work. In situations in which the reason for the absence is due to an employee’s serious health condition, the absence may be covered by the Family Medical Leave Act (FMLA) . Employees must practice and maintain personal hygiene to help ensure a healthy workplace. Strategies to accomplish this include: Washing hands to aid in keeping employees healthy and preventing the spread of illness to co-workers. All employees should adhere to the 

In [ ]:
!pip install openai



####Retrival pipeline 

In [ ]:
import openai
from typing import Dict, List, Any

def initialize_openai_client(api_key: str, base_url: str = 'https://api.ai.it.ufl.edu'):
    """
    Initialize OpenAI client with custom base URL.
    
    Args:
        api_key: Your OpenAI API key
        base_url: Custom base URL for API (default: UF API endpoint)
    
    Returns:
        OpenAI client object
    """
    try:
        client = openai.OpenAI(
            api_key=api_key,
            base_url=base_url
        )
        
        print('✓ OpenAI client created successfully')
        print(f'  Base URL: {base_url}')
        print(f'  Client type: {type(client)}')
        
        # Sanity checks
        has_chat = hasattr(client, 'chat')
        print(f'  Has chat attribute: {has_chat}')
        
        if has_chat:
            has_completions = hasattr(client.chat, 'completions')
            print(f'  Has completions attribute: {has_completions}')
        
        return client
    except Exception as e:
        print(f' Error initializing OpenAI client: {e}')
        return None

In [46]:
from typing import Dict, List, Any
def prepare_context_from_results(query_results: Dict[str, Any]) -> tuple:
    """
    Prepare context and metadata from ChromaDB query results.
    
    Args:
        query_results: Dictionary from ChromaDB query (with ids, documents, metadatas, distances)
    
    Returns:
        Tuple of (context_text, source_documents)
    """
    # Extract documents and metadata
    documents = query_results['documents'][0]
    metadatas = query_results['metadatas'][0]
    distances = query_results['distances'][0]
    
    # Build context text
    context_parts = []
    source_documents = []
    
    for i, (doc, metadata, distance) in enumerate(zip(documents, metadatas, distances), 1):
        # Add document to context with reference number
        context_parts.append(f"[Document {i}]")
        context_parts.append(f"Title: {metadata['title']}")
        context_parts.append(f"Type: {metadata['type']}")
        context_parts.append(f"Category: {metadata['category']}")
        context_parts.append(f"Content: {doc}")
        context_parts.append("")  # Empty line between documents
        
        # Store source information
        source_documents.append({
            'title': metadata['title'],
            'type': metadata['type'],
            'category': metadata['category'],
            'distance': distance,
            'link': metadata.get('link', 'N/A')
        })
    
    context_text = "\n".join(context_parts)
    
    return context_text, source_documents

In [ ]:
def query_openai_with_context(
    client,
    question: str,
    query_results: Dict[str, Any],
    model: str = "gpt-oss-120b",
    temperature: float = 0.1,
    max_tokens: int = 1000
) -> Dict[str, Any]:
    """
    Query OpenAI with context from ChromaDB results.
    
    Args:
        client: OpenAI client object
        question: User's question
        query_results: Results from ChromaDB query
        model: Model name (e.g., 'gpt-4', 'gpt-3.5-turbo')
        temperature: Sampling temperature (0-2, lower is more focused)
        max_tokens: Maximum tokens in response
    
    Returns:
        Dictionary with answer and source documents
    """
    try:
        # Prepare context from query results
        context_text, source_documents = prepare_context_from_results(query_results)
        
        # Create system prompt
        system_prompt = """You are a helpful assistant that answers questions based ONLY on the provided context documents.

IMPORTANT RULES:
1. Answer ONLY using information from the provided documents
2. Do NOT use any external knowledge or information not present in the documents
3. If the answer is not in the provided documents, say "I cannot find this information in the provided documents"
4. Cite which document(s) you used by referring to [Document N] format
5. Be accurate and concise in your response"""

        # Create user prompt with context
        user_prompt = f"""Context Documents:
{context_text}

Question: {question}

Please answer the question using ONLY the information from the context documents above. Do not use any external knowledge."""

        # Call OpenAI API
        print(f"\n Querying {model}...")
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        # Extract answer
        answer = response.choices[0].message.content
        
        # Prepare result
        result = {
            'question': question,
            'answer': answer,
            'source_documents': source_documents,
            'model': model,
            'temperature': temperature,
            'tokens_used': {
                'prompt': response.usage.prompt_tokens,
                'completion': response.usage.completion_tokens,
                'total': response.usage.total_tokens
            }
        }
        
        print("✓ Response received successfully")
        
        return result
        
    except Exception as e:
        print(f" Error querying OpenAI: {e}")
        return {
            'question': question,
            'answer': f"Error: {str(e)}",
            'source_documents': [],
            'model': model
        }

In [57]:
client = initialize_openai_client(
    api_key="sk-pgNP-BIHOtI8RPt-aC3Stg",
    base_url='https://api.ai.it.ufl.edu'
)


result = query_openai_with_context(
    client=client,
    question="research asistant polic?",
    query_results=results,
    model="gpt-oss-120b",
    temperature=0.1,
    max_tokens=1000
)

✓ OpenAI client created successfully
  Base URL: https://api.ai.it.ufl.edu
  Client type: <class 'openai.OpenAI'>
  Has chat attribute: True
  Has completions attribute: True

🤖 Querying gpt-oss-120b...
✓ Response received successfully


In [58]:
result

{'question': 'research asistant polic?',
 'answer': 'I cannot find this information in the provided documents.',
 'source_documents': [{'title': 'Working Safely and Maintaining Workplace Health Standards',
   'type': 'Policy',
   'category': 'Human Resources',
   'distance': 0.5599740743637085,
   'link': 'N/A'},
  {'title': 'Furlough Policy (includes UFF Faculty MOU)',
   'type': 'Policy',
   'category': 'Human Resources',
   'distance': 0.5678935050964355,
   'link': 'N/A'},
  {'title': 'Paid Family Leave',
   'type': 'Policy',
   'category': 'Human Resources',
   'distance': 0.5807972550392151,
   'link': 'N/A'}],
 'model': 'gpt-oss-120b',
 'temperature': 0.1,
 'tokens_used': {'prompt': 630, 'completion': 65, 'total': 695}}